# Patent Data

### Warning

This script has been modified to automate data download and environment setup. **Please ensure you're using your desired python environment before continuing**, and that you're running this from the GitHub repository root.

In [1]:
import os, sys, subprocess
# This does all the setup steps for the project - don't modify it
subprocess.run(['python3', '__init__.py'])

Installing required packages, creating necessary directories, and downloading the data...
PLEASE ENSURE YOU'RE USING YOUR DESIRED ENVIRONMENT!
Installing required packages...

--- DOWNLOADING RAW TABLES ---
[init]: g_patent.tsv already exists.
[init]: g_inventor_not_disambiguated.tsv already exists.
[init]: g_location_not_disambiguated.tsv already exists.
[init]: g_assignee_not_disambiguated.tsv already exists.
[init]: g_wipo_technology.tsv already exists.
[init]: All raw tables downloaded.
[init]: Initialization complete.


CompletedProcess(args=['python3', '__init__.py'], returncode=0)

In [2]:
import pandas as pd
from colorama import Fore, Style


In [3]:
# Paths (modified to be relative to the current working directory)
patents_g_p   = os.path.join(os.getcwd(), 'data', 'raw', 'g_patent.tsv')
applicant_g_p = os.path.join(os.getcwd(), 'data', 'raw', 'g_inventor_not_disambiguated.tsv')
location_g_p  = os.path.join(os.getcwd(), 'data', 'raw', 'g_location_not_disambiguated.tsv')
assignee_g_p  = os.path.join(os.getcwd(), 'data', 'raw', 'g_assignee_not_disambiguated.tsv')
wipo_g_p      = os.path.join(os.getcwd(), 'data', 'raw', 'g_wipo_technology.tsv')

In [4]:
# Dataload (~60s)

#patent information
patents_g = pd.read_csv(patents_g_p, sep='\t')

#applicant information (include multiple per patent doing seq = 0 should be primary)
applicant_g = pd.read_csv(applicant_g_p, sep='\t')

# raw location not disambiguated (e.g, no coordinates)
location_g = pd.read_csv(location_g_p, sep='\t')

#get organization
assignee_g = pd.read_csv(assignee_g_p, sep='\t')

#the sector and type of patent
wipo_g = pd.read_csv(wipo_g_p, sep='\t')

/var/folders/sx/6f330c0s1lx73cf7ntt5h3340000gp/T/ipykernel_96093/3429093518.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  patents_g = pd.read_csv(patents_g_p, sep='\t')


## Initial Parsing

All that's happening here is trimming down the `applicant_g` and `assignee_g` tables to ensure that each only contains the primary inventor/assignee organization for every patent. I don't love this approach, but it seems to be required if we continue to use Tableau.

In [5]:
# Simplifying inventor information to just primary inventor (~10s)
# Getting count of non-unique patent_id rows
non_uniques = applicant_g[applicant_g.duplicated(subset='patent_id', keep=False)]
print(f"{Style.BRIGHT}Duplicated patent_ids are duplicated because each patent/inventor combo has it's own row:{Style.RESET_ALL}")
print(f"Number of non-unique patent_id rows: {len(non_uniques)}.")

just_dupes = non_uniques[non_uniques.duplicated(subset='patent_id', keep='first')]
num_unique_patents = len(applicant_g['patent_id'].unique())
print(f"Removing all duplicates will result in {len(just_dupes)} fewer rows, but we will retain {num_unique_patents} patents (this doesn't change).")

# Value counts of inventor sequence for full data
print(f"\n{Style.BRIGHT}Value counts of inventor_sequence for the full data:{Style.RESET_ALL}")
print(applicant_g['inventor_sequence'].value_counts().sort_index())

Duplicated patent_ids are duplicated because each patent/inventor combo has it's own row:
Number of non-unique patent_id rows: 19235677.
Removing all duplicates will result in 13616741 fewer rows, but we will retain 8979043 patents (this doesn't change).

Value counts of inventor_sequence for the full data:
inventor_sequence
0      8979043
1      5618936
2      3455401
3      1985134
4      1083263
        ...   
128          7
129          7
130          2
131          2
132          2
Name: count, Length: 133, dtype: int64


The simplest way to deal with this is actually just to drop all patent rows where `inventor_sequence` isn't $0$. This won't reduce our overall number of unique patents, but will reduce our ability to trace connections between inventors working on the same projects, potentially eliminating up to $132$ collaborators in $2$ cases.

In [6]:
# Drop all rows with inventor_sequence > 0
applicant_g = applicant_g[applicant_g['inventor_sequence'] == 0].drop(columns=['inventor_sequence', 'deceased_flag'])
print(f"{Style.DIM}applicant_g{Style.NORMAL} now has {len(applicant_g)} rows, matching our patent count.")

applicant_g now has 8979043 rows, matching our patent count.


We've actually got to do the same thing for the `assignee_g` table, as apparently Tableau won't let us include lists for this stuff. We're losing a ton of data at this step, so it may be worth looking into fixing this on the Tableau side instead of removing all but one inventor and assignee.

In [7]:
# Simplifying assignee information to just primary assignee (~10s)
# Getting count of non-unique patent_id rows
non_uniques = assignee_g[assignee_g.duplicated(subset='patent_id', keep=False)]
print(f"\n{Style.BRIGHT}Duplicated patent_ids are duplicated because each patent/assignee combo has it's own row:{Style.RESET_ALL}")

just_dupes = non_uniques[non_uniques.duplicated(subset='patent_id', keep='first')]
num_unique_patents = len(assignee_g['patent_id'].unique())
print(f"Removing all duplicates will result in {len(just_dupes)} fewer rows, but we will retain {num_unique_patents} patents.")

# Value counts of assignee seq for full data
print(f"\n{Style.BRIGHT}Value counts of assignee_sequence for the full data:{Style.RESET_ALL}")
print(assignee_g['assignee_sequence'].value_counts().sort_index())



Duplicated patent_ids are duplicated because each patent/assignee combo has it's own row:
Removing all duplicates will result in 293334 fewer rows, but we will retain 7999864 patents.

Value counts of assignee_sequence for the full data:
assignee_sequence
0     7999864
1      259642
2       25797
3        4903
4        1503
5         667
6         285
7         173
8         122
9          93
10         56
11         39
12         23
13          6
14          4
15          2
16          1
17          1
18          1
19          1
20          1
21          1
22          1
23          1
24          1
25          1
26          1
27          1
28          1
29          1
30          1
31          1
32          1
33          1
34          1
Name: count, dtype: int64


In [8]:
# Drop all rows with assignee_sequence > 0
assignee_g = assignee_g[assignee_g['assignee_sequence'] == 0].drop(columns=['assignee_sequence', 'assignee_type'])
print(f"{Style.DIM}assignee_g{Style.NORMAL} now has {len(assignee_g)} rows. Note that this {Style.BRIGHT}doesn't{Style.NORMAL} match our patent count, as not every patent has an assignee.")

assignee_g now has 7999864 rows. Note that this doesn't match our patent count, as not every patent has an assignee.


## Visualization

We now have the following 5 tables that we need to combine into one.

#### Patent

In [9]:
patents_g.head(5)

,patent_id,patent_type,patent_date,patent_title,wipo_kind,num_claims,withdrawn,filename
0,10000000,utility,2018-06-19,Coherent LADAR using intra-pixel quadrature de...,B2,20,0,ipg180619.xml
1,10000001,utility,2018-06-19,Injection molding machine and mold thickness c...,B2,12,0,ipg180619.xml
2,10000002,utility,2018-06-19,Method for manufacturing polymer film and co-e...,B2,9,0,ipg180619.xml
3,10000003,utility,2018-06-19,Method for producing a container from a thermo...,B2,18,0,ipg180619.xml
4,10000004,utility,2018-06-19,"Process of obtaining a double-oriented film, c...",B2,6,0,ipg180619.xml


#### Inventor (Applicant)

In [10]:
applicant_g.head()

,patent_id,inventor_id,raw_inventor_name_first,raw_inventor_name_last,rawlocation_id
0,D1006496,fl:we_ln:jiang-128,Wenjing,Jiang,30zgod902k0u495w9b2sb8xk9
2,6584128,fl:ri_ln:kroeger-1,Richard,Kroeger,o2ema6bl3kkh6iwziprbzu4m9
3,4789863,fl:th_ln:bush-1,Thomas A.,Bush,tkz70bmoqx88n1lfz3fl657lh
6,D474886,fl:th_ln:fleming-4,Thomas W.,Fleming,cmf9hebugybql8ufffo47uwcz
8,7646155,fl:sa_ln:woods-2,Samuel G,Woods,omi6wqlrblholsssk9qx0dz5b


#### Organization (Assignee)

In [11]:
assignee_g.head()

,patent_id,assignee_id,raw_assignee_individual_name_first,raw_assignee_individual_name_last,raw_assignee_organization,rawlocation_id
0,4488683,0e9bf1f9-0c30-4255-b12f-54b5ee54b8e4,NaN,NaN,Metal Works Ramat David,b44f6bf0-1f14-4b25-9ab6-06945ff1e8e1
1,11872626,7336116d-16f3-4cb8-ad9c-97f8e8ac2c2a,NaN,NaN,"DIVERGENT TECHNOLOGIES, INC.",8zd6qm7aatxho9jcoihh78un1
2,5856666,cf1ecab5-bf3a-4dc8-a335-85c98035e250,NaN,NaN,U.S. Philips Corporation,orskbf54s58e97lkmw8na5rpx
3,5204210,34dbb232-405f-4c13-985f-da2d62f47c7c,NaN,NaN,Xerox Corporation,mue862v5lcjdhzqqk86ei75kj
5,D397841,b4894b91-86d9-49e2-b37c-cb4fd6da5327,NaN,NaN,"adidas, AG",83c2755a-df62-4f4f-8509-43b9dcfeb038


#### Location

For each `rawlocation_id` in the `applicant_g` and `assignee_g` tables, we'll need to merge to get inventor location and assignee location.

In [12]:
location_g.head(5)

,rawlocation_id,location_id,raw_city,raw_state,raw_country
0,000005mtrirpdyrtlkfbffj0e,1d2251c8-16c8-11ed-9b5f-1234bde3cd05,Kanagawa,NaN,JP
1,00006fjnoq057no2s4bse374r,439af3dd-16c8-11ed-9b5f-1234bde3cd05,Dallas,TX,US
2,0000bztanu9rrtm943i8a7wry,8d09bc30-16c8-11ed-9b5f-1234bde3cd05,Osaka,NaN,JP
3,0000c07c-672a-48d9-a3bb-71c439ac17b3,280c9de4-16c8-11ed-9b5f-1234bde3cd05,NaN,NaN,DE
4,0000dpr8zljw34um291yal5ym,cf4944e5-16c7-11ed-9b5f-1234bde3cd05,Sugar Land,TX,US


### Wipo Technology

In [13]:
wipo_g.head(5)

,patent_id,wipo_field_sequence,wipo_field_id,wipo_sector_title,wipo_field_title
0,10000000,0,10,Instruments,Measurement
1,10000001,0,29,Mechanical engineering,Other special machines
2,10000001,1,12,Instruments,Control
3,10000002,0,21,Chemistry,"Surface technology, coating"
4,10000002,1,29,Mechanical engineering,Other special machines


We'll want the `wipo_sector_title` and `wipo_field_title` columns from this after matching on `patent_id`.

## Merging Data

In [14]:
# Assure all ids are string type is there is no issue when merging
patents_g.loc[:, 'patent_id']       = patents_g['patent_id'].astype('string')
applicant_g.loc[:, 'patent_id']     = applicant_g['patent_id'].astype('string')
assignee_g.loc[:, 'patent_id']      = assignee_g['patent_id'].astype('string')
wipo_g.loc[:, 'patent_id']          = wipo_g['patent_id'].astype('string')
location_g.loc[:, 'rawlocation_id'] = location_g['rawlocation_id'].astype('string')

# ignore the warning

/var/folders/sx/6f330c0s1lx73cf7ntt5h3340000gp/T/ipykernel_96093/2422595300.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<StringArray>
['10000000', '10000001', '10000001', '10000002', '10000002', '10000002',
 '10000003', '10000004', '10000005', '10000006',
 ...
  '9999994',  '9999994',  '9999995',  '9999996',  '9999996',  '9999997',
  '9999997',  '9999998',  '9999998',  '9999999']
Length: 12030035, dtype: string' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  wipo_g.loc[:, 'patent_id']          = wipo_g['patent_id'].astype('string')


### Merging Location of Assignee & Inventor

In [15]:
# Simplifying the table to just what we need (very few first/last names)
assignee_g = assignee_g[['patent_id', 'raw_assignee_organization', 'rawlocation_id']]
assignee_g.loc[:, 'patent_id'] = assignee_g['patent_id'].astype('string')
assignee_g.loc[:, 'rawlocation_id'] = assignee_g['rawlocation_id'].astype('string')

In [16]:
# Merge assignee_g with locations to get assignee_city, assignee_state, assignee_country (~20s)
assignee_g = pd.merge(assignee_g, location_g, on="rawlocation_id", how="left")
assignee_g = assignee_g.rename(columns={
    'raw_city': 'assignee_city', 
    'raw_state': 'assignee_state', 
    'raw_country': 'assignee_country', 
    'raw_assignee_organization': 'assignee'
})
assignee_g.drop(columns=['rawlocation_id', 'location_id'], inplace=True)
assignee_g.head()

,patent_id,assignee,assignee_city,assignee_state,assignee_country
0,4488683,Metal Works Ramat David,NaN,NaN,IL
1,11872626,"DIVERGENT TECHNOLOGIES, INC.",Los Angeles,CA,US
2,5856666,U.S. Philips Corporation,New York,NY,US
3,5204210,Xerox Corporation,Stamford,CT,US
4,D397841,"adidas, AG",NaN,NaN,DE


We also need to merge the location and the inventors, so we can essentially have a **start** (primary inventor) location and **end** (primary assignee) location for each patent.

In [17]:
# Simplifying the table to just what we need
applicant_g = applicant_g.drop(columns=['inventor_id'])
applicant_g.loc[:, 'patent_id'] = applicant_g['patent_id'].astype('string')
applicant_g.loc[:, 'rawlocation_id'] = applicant_g['rawlocation_id'].astype('string')

In [18]:
# Merge applicant_g with locations to get inventor_city, inventor_state, inventor_country (~20s)
applicant_g = pd.merge(applicant_g, location_g, on="rawlocation_id", how="left")
applicant_g = applicant_g.rename(columns={
    'raw_city': 'inventor_city', 
    'raw_state': 'inventor_state', 
    'raw_country': 'inventor_country',
    'raw_inventor_name_first': 'inventor_firstname',
    'raw_inventor_name_last': 'inventor_lastname'
})
applicant_g.drop(columns=['rawlocation_id', 'location_id'], inplace=True)
applicant_g.head()

,patent_id,inventor_firstname,inventor_lastname,inventor_city,inventor_state,inventor_country
0,D1006496,Wenjing,Jiang,Guizhou,NaN,CN
1,6584128,Richard,Kroeger,NaN,NaN,NaN
2,4789863,Thomas A.,Bush,NaN,NaN,NaN
3,D474886,Thomas W.,Fleming,San Diego,CA,US
4,7646155,Samuel G,Woods,Bel Air,MD,US


In [19]:
# Now we merge both of those together based on patent_id
locations_df = pd.merge(applicant_g, assignee_g, on = 'patent_id', how = 'left')
locations_df.head()

,patent_id,inventor_firstname,inventor_lastname,inventor_city,inventor_state,inventor_country,assignee,assignee_city,assignee_state,assignee_country
0,D1006496,Wenjing,Jiang,Guizhou,NaN,CN,NaN,NaN,NaN,NaN
1,6584128,Richard,Kroeger,NaN,NaN,NaN,"Scientific-Atlanta, Inc.",Lawrenceville,GA,US
2,4789863,Thomas A.,Bush,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,D474886,Thomas W.,Fleming,San Diego,CA,US,"Bacou-Dalloz USA Safety, Inc.",San Diego,CA,US
4,7646155,Samuel G,Woods,Bel Air,MD,US,Balck & Decker Inc.,Newark,DE,US


In [21]:
# Counting number of missing first and last names
no_f = sum(locations_df['inventor_firstname'].isna())
no_l = sum(locations_df['inventor_lastname'].isna())

print(f'Number of missing first names: {no_f}')
print(f'Number of missing last names: {no_l}')

Number of missing first names: 198
Number of missing last names: 1


In [ ]:
missing_in_location_assignee = patents_g[~patents_g['patent_id'].isin(locations_df['patent_id'])]
print("Rows in patents_g with no match in location_assignee:", len(missing_in_location_assignee))

missing_in_patents_g = locations_df[~locations_df['patent_id'].isin(patents_g['patent_id'])]
print("Rows in location_assignee with no match in patents_g:", len(missing_in_patents_g))


Rows in patents_g with no match in location_assignee: 1087
Rows in location_assignee with no match in patents_g: 0


### Adding Patent Data
After this step, all we need is the WIPO categories for each patent. We'll also go through here and drop all rows with `patent_type != 'utility'`, as $>99.97%$ of the data falls into the 'utility' category, and for consistency we might as well drop the $~50$ rows not categorized as such.

In [22]:
# Almost done combining (~12s)
df_no_wipo = pd.merge(patents_g, locations_df, on='patent_id', how='left')
df_no_wipo = df_no_wipo[df_no_wipo['patent_type'] == "utility"].drop(columns=['patent_type'])
df_no_wipo.head()

,patent_id,patent_date,patent_title,wipo_kind,num_claims,withdrawn,filename,inventor_firstname,inventor_lastname,inventor_city,inventor_state,inventor_country,assignee,assignee_city,assignee_state,assignee_country
0,10000000,2018-06-19,Coherent LADAR using intra-pixel quadrature de...,B2,20,0,ipg180619.xml,Joseph,Marron,Manhattan Beach,CA,US,Raytheon Company,Waltham,MA,US
1,10000001,2018-06-19,Injection molding machine and mold thickness c...,B2,12,0,ipg180619.xml,Sun-Woo,Lee,Gunpo-si,NaN,KR,LS MTRON LTD.,"Anyang-si, Gyeonggi-Do",NaN,KR
2,10000002,2018-06-19,Method for manufacturing polymer film and co-e...,B2,9,0,ipg180619.xml,Yun-Jo,Kim,Yongin-si,NaN,KR,"KOLON INDUSTRIES, INC.",Gwacheon-si,NaN,KR
3,10000003,2018-06-19,Method for producing a container from a thermo...,B2,18,0,ipg180619.xml,Guido,Bergmann,St. Augustin,NaN,DE,KAUTEX TEXTRON GMBH & CO. KG,Bonn,NaN,DE
4,10000004,2018-06-19,"Process of obtaining a double-oriented film, c...",B2,6,0,ipg180619.xml,Miguel Jorge,Zubiria Elizondo,Monterrey,NaN,MX,ZUBEX INDUSTRIAL SA DE CV,Monterrey,NaN,MX


### Getting Values with only Valid location

In [ ]:
# Explore if there are any state or city information for rows missing countries for either assignee or inventor
def explore_missing_locs(colprefix: str):
    missing_countries = df_no_wipo[df_no_wipo[f'{colprefix}_country'].isna()]
    non_na = missing_countries[missing_countries[f'{colprefix}_state'].notna() | missing_countries[f'{colprefix}_city'].notna()]

    print(f"Number of rows with missing country for {colprefix} but non-missing state or city: {len(non_na)}")
    if len(missing_countries) != 0:
        print(f"This is {len(non_na) / len(missing_countries) * 100:.2f}% of the rows with missing countries, and {len(non_na) / len(df_no_wipo) * 100:.2f}% of the total rows.")
    print(f"This is too few to be useful, so we're dropping these.")

explore_missing_locs('assignee')
explore_missing_locs('inventor')

df_no_wipo = df_no_wipo.dropna(subset=['assignee_country'])
df_no_wipo = df_no_wipo.dropna(subset=['inventor_country'])

print(f"{Style.BRIGHT}\nCurrent length of the dataframe (# of unique patents with inventor and assignee locations) = {len(df_no_wipo)}.{Style.RESET_ALL}")

Number of rows with missing country for assignee but non-missing state or city: 0
This is too few to be useful, so we're dropping these.
Number of rows with missing country for inventor but non-missing state or city: 0
This is too few to be useful, so we're dropping these.
Current length of the dataframe (# of unique patents with inventor and assignee locations) = 7330807.


### WIPO Type and Sector

Per-patent, extract more detailed sector/industry categories from the wipo table.

In [ ]:
# Merge to get WIPO type for each patent (~10s)
df = pd.merge(df_no_wipo, wipo_g, on='patent_id', how='left')
df.head()

,patent_id,patent_date,patent_title,wipo_kind,num_claims,withdrawn,filename,inventor_firstname,inventor_lastname,inventor_city,inventor_state,inventor_country,assignee,assignee_city,assignee_state,assignee_country,wipo_field_sequence,wipo_field_id,wipo_sector_title,wipo_field_title
0,10000000,2018-06-19,Coherent LADAR using intra-pixel quadrature de...,B2,20,0,ipg180619.xml,Joseph,Marron,Manhattan Beach,CA,US,Raytheon Company,Waltham,MA,US,0.0,10.0,Instruments,Measurement
1,10000001,2018-06-19,Injection molding machine and mold thickness c...,B2,12,0,ipg180619.xml,Sun-Woo,Lee,Gunpo-si,NaN,KR,LS MTRON LTD.,"Anyang-si, Gyeonggi-Do",NaN,KR,0.0,29.0,Mechanical engineering,Other special machines
2,10000001,2018-06-19,Injection molding machine and mold thickness c...,B2,12,0,ipg180619.xml,Sun-Woo,Lee,Gunpo-si,NaN,KR,LS MTRON LTD.,"Anyang-si, Gyeonggi-Do",NaN,KR,1.0,12.0,Instruments,Control
3,10000002,2018-06-19,Method for manufacturing polymer film and co-e...,B2,9,0,ipg180619.xml,Yun-Jo,Kim,Yongin-si,NaN,KR,"KOLON INDUSTRIES, INC.",Gwacheon-si,NaN,KR,0.0,21.0,Chemistry,"Surface technology, coating"
4,10000002,2018-06-19,Method for manufacturing polymer film and co-e...,B2,9,0,ipg180619.xml,Yun-Jo,Kim,Yongin-si,NaN,KR,"KOLON INDUSTRIES, INC.",Gwacheon-si,NaN,KR,1.0,29.0,Mechanical engineering,Other special machines


In [ ]:
# Drop the few remaining columns we don't need
df = df.drop(columns=['wipo_field_sequence', 'wipo_field_id', 'wipo_kind'])

# Sort by date, and drop all before 2001-01-01
df['patent_date'] = pd.to_datetime(df['patent_date'])
df = df[df['patent_date'] >= '2001-01-01'].sort_values('patent_date')

# Assuming we're using invention location as the location predictor:
df = df[df['inventor_country' != 'US']]

# Ensure safe formatting for the .tsv (wipes out tabs in all str cols)
for col in df.columns:
    if isinstance(df[col].dtype, object):
        df[col] = df[col].str.replace('\t', ' ')

df.head()

,patent_id,patent_date,patent_title,num_claims,withdrawn,filename,inventor_firstname,inventor_lastname,inventor_city,inventor_state,inventor_country,assignee,assignee_city,assignee_state,assignee_country,wipo_sector_title,wipo_field_title
5844932,6169710,2001-01-02,Optical disc recording/reproducing method and ...,11,0,pftaps20010102_wk01.zip,Toru,Arai,Tokyo,NaN,JPX,NEC Corporation,NaN,NaN,JP,Electrical engineering,Audio-visual technology
5844395,6169280,2001-01-02,Light selective element for imaging applications,2,0,pftaps20010102_wk01.zip,Edward J.,Bawolek,Chandler,AZ,US,Intel Corporation,Santa Clara,CA,US,Electrical engineering,Semiconductors
5844396,6169281,2001-01-02,Apparatus and method for determining side wall...,28,0,pftaps20010102_wk01.zip,Dong,Chen,Boca Raton,FL,US,International Business Machines Corporation,Armonk,NY,US,Instruments,Measurement
5844397,6169282,2001-01-02,Defect inspection method and apparatus therefor,32,0,pftaps20010102_wk01.zip,Shunji,Maeda,Yokohama,NaN,JPX,"Hitachi, Ltd.",Tokyo,NaN,JPX,Electrical engineering,Computer technology
5844398,6169282,2001-01-02,Defect inspection method and apparatus therefor,32,0,pftaps20010102_wk01.zip,Shunji,Maeda,Yokohama,NaN,JPX,"Hitachi, Ltd.",Tokyo,NaN,JPX,Instruments,Measurement


In [28]:
print(f"{Style.BRIGHT}{Fore.MAGENTA}This leaves us with a total of {len(df)} patents with full information since January 1st, 2001.{Style.RESET_ALL}")

This leaves us with a total of 8478512 patents with full information since January 1st, 2001.


In [29]:
dest_path = os.path.join(os.getcwd(), 'data', 'processed_patents.tsv')
df.to_csv(dest_path, sep='\t', index=False)